# Cross-validation (CV, 交叉驗證)

目的 : 防止模型過於複雜而引起過度擬合 (overfitting)

交叉驗證主要分為以下幾類
1.  K-folder CV:

    將訓練集分成 k 個子集，每個子集都做一次測試集，其他子集做訓練集。最後將 k 次的結果(AUC或Acc)平均，則得到 CV score，以此當作結果。
    
2.  KK-folder CV

    和 K-folder CV 相似，但是此把 k 個子集平均分成 2 個子集(s0,s1)。用 s0 當訓練 s1 當測試; s1 當訓練 s0 當測試。可以想像成把資料分成 k 份後再做一次 k-folder CV。
    
3.  least-one-out CV (LOOCV)

    依照要本數去決定 k-folder 的 k 值。例如假設有 n 個樣本，則使用 k-folder CV，這裡 k = n 。

    優點 :
    1.  每一次 fit model 時，每個樣本幾乎都被使用到，所以 model 會最接近母體樣本的分佈。
    2.  實驗中沒有隨機因素會影響結果

    缺點 :
    1.  相較前兩者，計算成本較高。
    

參考資料
1.   [[Day29]機器學習：交叉驗證！](https://ithelp.ithome.com.tw/articles/10197461)
2.   [机器学习中的交叉验证（cross-validation）](https://blog.csdn.net/lhx878619717/article/details/49079785)